In [ ]:
import pandas as pd

df = pd.read_parquet("data/intermediate/m5_scaled.parquet")

In [ ]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering

db = DBSCAN(eps=0.5, min_samples=5, metric="euclidean")
db_labels = db.fit_predict(df_scaled)

In [1]:
df_scaled = df["y_scaled"].values.reshape(-1, 1)
agglomerative = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5)
agglomerative_labels = agglomerative.fit_predict(df_scaled)

NameError: name 'df' is not defined

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN, AffinityPropagation, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn_extra.cluster import KMedoids
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

# Define clustering functions


def perform_dbscan(df_scaled):
    db = DBSCAN(eps=0.5, min_samples=5, metric="euclidean")
    db_labels = db.fit_predict(df_scaled)
    return db_labels


def perform_agglomerative_clustering(df_scaled):
    agglomerative = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5)
    agglomerative_labels = agglomerative.fit_predict(df_scaled)
    return agglomerative_labels


# def perform_k_medoids(df_scaled):
#     kmedoids = KMedoids(n_clusters=3, metric="euclidean")
#     kmedoids_labels = kmedoids.fit_predict(df_scaled)
#     return kmedoids_labels


def perform_gmm(df_scaled):
    gmm = GaussianMixture(n_components=3)
    gmm_labels = gmm.fit_predict(df_scaled)
    return gmm_labels


def perform_k_shape(df_scaled):
    ts = TimeSeriesScalerMeanVariance()
    df_scaled = ts.fit_transform(df_scaled)
    kshape = KShape(n_clusters=3, verbose=True)
    kshape_labels = kshape.fit_predict(df_scaled)
    return kshape_labels


def perform_affinity_propagation(df_scaled):
    affprop = AffinityPropagation(damping=0.9)
    affprop_labels = affprop.fit_predict(df_scaled)
    return affprop_labels


# Main function to cluster and return results


def cluster_time_series(df_scaled, unique_id_col="unique_id", y_scaled_col="y_scaled"):

    # Perform clustering using all methods
    dbscan_labels = perform_dbscan(df_scaled)
    agglomerative_labels = perform_agglomerative_clustering(df_scaled)
    # kmedoids_labels = perform_k_medoids(df_scaled)
    gmm_labels = perform_gmm(df_scaled)
    kshape_labels = perform_k_shape(df_scaled)
    affprop_labels = perform_affinity_propagation(df_scaled)

    # Add results back to the dataframe
    df["dbscan_labels"] = dbscan_labels
    df["agglomerative_labels"] = agglomerative_labels
    # df['kmedoids_labels'] = kmedoids_labels
    df["gmm_labels"] = gmm_labels
    df["kshape_labels"] = kshape_labels
    df["affprop_labels"] = affprop_labels

    # Optionally compute silhouette scores for evaluation
    silhouette_scores = {
        "DBSCAN": silhouette_score(df_scaled, dbscan_labels),
        "Agglomerative": silhouette_score(df_scaled, agglomerative_labels),
        # 'K-Medoids': silhouette_score(df_scaled, kmedoids_labels),
        "GMM": silhouette_score(df_scaled, gmm_labels),
        "K-Shape": silhouette_score(df_scaled, kshape_labels),
        "Affinity Propagation": silhouette_score(df_scaled, affprop_labels),
    }

    # Print the silhouette scores for evaluation
    for method, score in silhouette_scores.items():
        print(f"Silhouette Score for {method}: {score:.4f}")

    return df, silhouette_scores


# Function to visualize time series clusters using Plotly


def plot_clusters(df, label_col, n_clusters=6):
    # Filter random samples
    unique_labels = np.unique(df[label_col])
    sample_clusters = np.random.choice(unique_labels, size=n_clusters, replace=False)

    fig = go.Figure()

    for cluster in sample_clusters:
        cluster_data = df[df[label_col] == cluster]
        fig.add_trace(
            go.Scatter(
                x=np.arange(len(cluster_data)),
                y=cluster_data["y_scaled"],
                mode="lines",
                name=f"Cluster {cluster}",
            )
        )

    fig.update_layout(
        title=f"Random {n_clusters} Time Series from {label_col} Clusters",
        xaxis_title="Time",
        yaxis_title="Scaled Value",
        showlegend=True,
    )
    fig.show()


# Example usage with a dummy DataFrame (replace with your actual data)
df = pd.read_parquet("data/intermediate/m5_scaled.parquet")  # Replace with your dataset
df_with_labels, silhouette_scores = cluster_time_series(df)

# Display the result with cluster labels
print(df_with_labels)

# Visualize the time series of each cluster
for label_col in [
    "dbscan_labels",
    "agglomerative_labels",
    "kmedoids_labels",
    "gmm_labels",
    "kshape_labels",
    "affprop_labels",
]:
    print(f"\nVisualizing {label_col}")
    plot_clusters(df_with_labels, label_col)

/home/pranav-pc/projects/ts/nbs/clustering/lab/lib/python3.12/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/pranav-pc/.pyenv/versions/3.12.9/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/pranav-pc/.pyenv/versions/3.12.9/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  Fil

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [2]:
pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-learn-extra: filename=scikit_learn_extra-0.3.0-cp312-cp312-linux_x86_64.whl size=2525905 sha256=dc8fd5bbf932df0ceefbcfcdb9ec33dee078ab14c209bf16a6d38c1a2260ba6c
  Stored in directory: /home/pranav-pc/.cache/pip/wheels/17/4d/c3/c6d5d563c1bf8146d059d63be3678abc2f2801fba0aaf5f0b8
Successfully built scikit-learn-extra

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
